## 분류 분석
- 실제 분류와 예측 분류가 얼마나 일치했는가를 기반으로 알고리즘 성능을 평가

### 정확도 
- 실제 데이터에서 예측 데이터가 얼마나 같은지 판단하는 지표
- 데이터 구성에 따라 머신러닝 모델의 성능을 왜곡할 가능성이 존재

### 혼동행렬
- 이진 분류의 예측 오류가 얼마이고 어떠한 유형의 오류가 발생하고 있는지 나타내는 지표
- 4분면 행렬에서 실제 라젤 클래스 값과 예측 라벨 클래스 값이 어떠한 유형을 가지고 매핑이 되는지 나타냄
    - TN : 예측 값이 Negative(0), 실제 값도 Negative(0)
    - FP : 예측 값이 Positive(1), 실제 값이 Negative(0)
    - FN : 예측 값이 Negative(0), 실제 값이 Positive(1)
    - TP : 예측 값이 Positive(1), 실제 값도 Positive(1)

### 정밀도 
- Positive로 예측한 것들 중 실제로도 Positive인 것들의 비율
- Positive 예측 성능을 더욱 정밀하게 측정하기 위한 평가지표
- 양성 예측도라 불린다. 
- 정밀도가 상대적인 중요성을 가지는 경우는 실제 Negative인 데이터를 Positive로 잘못 예측했을 때 업무상 큰 영향이 발생할 때

### 재현율
- 실제 Positive인 것들 중 Positive로 예측한 비율

### F1 스코어
- 정밀도와 재현율을 결합한 분류 성능 지표
- 실제 Positive인 것들 중 Positive로 예측한 것들의 비율
- 정밀도와 재현율이 어느 한쪽으로 치우치지 않고 적절한 조화를 이루는 경우에 상대적으로 높은 수치를 나타냄

## ROC 곡선과 AUC 스코어
- ROC 곡선은 FPR(False Positive Rate)이 변할 때 TPR(True Positivie Rate)이 변하는 것을 나타내는 곡선
- AUC 스코어는 ROC 곡선 아래 부분의 면적 값을 분류하는 성능 지표로서 사용

In [3]:
from sklearn.datasets import load_iris
import pandas as pd

In [4]:
iris = load_iris()
## iris.data는 독립변수만으로 된 데이터
iris_dt = iris.data
## iris.target 종속변수 값
iris_label = iris.target

df = pd.DataFrame(data = iris_dt, columns=iris.feature_names)
df['Species'] = iris_label

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
## 데이터의 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    iris_dt, 
    iris_label, 
    test_size= 0.2, 
    random_state=0, 
    stratify = iris_label
)

In [6]:
## 데이터의 전처리 
## 아이리스 데이터는 독립변수들이 모두 연속형으로 단위가 동일하기 때문에 표준화 정규화 등의 전처리가 필요 X
## 결측치 확인
df.isna().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
Species              0
dtype: int64

In [7]:
## 모델 학습 
## 의사결정나무를 사용해서 분류 분석 수행
## 트리의 깊이는 5, 3, 1로 설정한 의사결정나무 모델 세가지를 생성
from sklearn.tree import DecisionTreeClassifier
dtree_clf_5 = DecisionTreeClassifier(max_depth=5, random_state=100)
dtree_clf_3 = DecisionTreeClassifier(max_depth=3, random_state=100)
dtree_clf_1 = DecisionTreeClassifier(max_depth=1, random_state=100)

In [8]:
## 검증 평가 10회
from sklearn.model_selection import cross_val_score
import numpy as np
scores = cross_val_score(dtree_clf_5, x_train, y_train, scoring='accuracy', cv = 10)
print('교차검증 정확도 :', np.round(scores, 3))
print('평균 검증 정확도 :', np.round(np.mean(scores), 4))

교차검증 정확도 : [0.917 1.    0.917 1.    1.    0.833 1.    0.917 1.    0.833]
평균 검증 정확도 : 0.9417


In [9]:
scores = cross_val_score(dtree_clf_3, x_train, y_train, scoring='accuracy', cv = 10)
print('교차검증 정확도 :', np.round(scores, 3))
print('평균 검증 정확도 :', np.round(np.mean(scores), 4))

교차검증 정확도 : [0.917 1.    0.917 0.917 1.    0.833 1.    0.917 0.917 0.833]
평균 검증 정확도 : 0.925


In [10]:
scores = cross_val_score(dtree_clf_1, x_train, y_train, scoring='accuracy', cv = 10)
print('교차검증 정확도 :', np.round(scores, 3))
print('평균 검증 정확도 :', np.round(np.mean(scores), 4))

교차검증 정확도 : [0.667 0.667 0.667 0.667 0.667 0.667 0.667 0.667 0.667 0.667]
평균 검증 정확도 : 0.6667


In [11]:
## 교차 검증 결과 : 트리의 깊이가 5인 경우 가장 높은 정확도를 나타냄
## 해당 모델로 평가 데이터를 적용하여 알고리즘 성능 평가를 수행 후 예측 값을 지정
dtree_clf_5.fit(x_train, y_train)
pred = dtree_clf_5.predict(x_test)
from sklearn.metrics import accuracy_score
print("의사결정나무(교차검증 후) 예측 정확도 : {0: .5f}".format(accuracy_score(y_test, pred)))

의사결정나무(교차검증 후) 예측 정확도 :  0.96667


In [12]:
pred = pd.DataFrame(pred, columns=['Pred Species'])
pred.head()

,Pred Species
0,0
1,1
2,0
3,2
4,0


In [13]:
actual = pd.DataFrame(y_test, columns=['Actual Species'])
actual.head()

,Actual Species
0,0
1,1
2,0
3,2
4,0


In [14]:
classify_result = pd.concat([actual, pred], axis=1)
classify_result.to_csv('clf_result.csv', index=False, encoding='utf-8')
classify_result.head()

,Actual Species,Pred Species
0,0,0
1,1,1
2,0,0
3,2,2
4,0,0
